In [3]:
%load_ext autoreload
%autoreload 2

import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Concatenate, Dense, Flatten, Input, Reshape
from keras.models import Sequential, Model
from PIL import Image
from pprint import pprint

from keras_preprocessing.image import DataFrameIterator, ImageDataGenerator


pixel_val = 1
np_filenames, img_filenames, np_arrays = [], [], []
for i in range(10):
    np_filename = '/tmp/{}.npy'.format(i)
    img_filename = '/tmp/{}.jpg'.format(i)
    array = pixel_val * np.random.randint(0, 255, size=(2, 2)).astype('float32')
    np.save(np_filename, array[..., None])
    plt.imsave(img_filename, array)
    
    np_filenames.append(np_filename)
    img_filenames.append(img_filename)
    
df = pd.DataFrame({'img_path': img_filenames, 'img_np_path': np_filenames})
df['regression'] = np.random.randint(1, 10, size=len(df))
df['binary'] = random.sample(['dog', 'cat'] * 30, len(df))
df['multi_class'] = random.sample(['dog', 'cat', 'horse'] * 30, len(df))
df['multi_label'] = random.sample(['dog', 'cat', ['dog'], ['cat'], ['cat', 'dog'], ['dog', 'cat']] * 10, len(df))
df['img_path_extra'] = img_filenames
df['regression_extra'] = np.random.randint(10, 100, size=len(df))

df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,img_path,img_np_path,regression,binary,multi_class,multi_label,img_path_extra,regression_extra
0,/tmp/0.jpg,/tmp/0.npy,5,dog,dog,[dog],/tmp/0.jpg,72
1,/tmp/1.jpg,/tmp/1.npy,6,cat,cat,"[dog, cat]",/tmp/1.jpg,71
2,/tmp/2.jpg,/tmp/2.npy,8,dog,horse,"[dog, cat]",/tmp/2.jpg,33
3,/tmp/3.jpg,/tmp/3.npy,7,dog,dog,[dog],/tmp/3.jpg,32
4,/tmp/4.jpg,/tmp/4.npy,9,dog,dog,"[dog, cat]",/tmp/4.jpg,33


### Multi-input with regression

without data augmentation

In [5]:
df_iter = DataFrameIterator(
    df, 
    input_columns=['img_path', 'img_path_extra'],
    output_columns='regression',
    image_size=(2, 2),
    batch_size=2,
    color_mode='grayscale'
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

input_1 = Input(shape=(2, 2, 1))
input_2 = Input(shape=(2, 2, 1))
x_1 = Flatten()(input_1)
x_2 = Flatten()(input_2)
x = Concatenate()([x_1, x_2])
out = Dense(1)(x)
model = Model([input_1, input_2], out)
model.compile('adam', 'mse')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg',
             '/tmp/8.jpg',
             '/tmp/9.jpg']},
 {'column': 'img_path_extra',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg',
             '/tmp/8.jpg',
             '/tmp/9.jpg']}]

Outputs:
[{'column': 'regression',
  'mode': None,
  'values': [5, 6, 8, 7, 9, 3, 2, 1, 7, 6]}]


W1009 17:49:06.892046 139933432076096 deprecation_wrapper.py:119] From /home/rodrigo/anaconda3/envs/keras-dev/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/1
5/5 [==============================] - 0s 22ms/step - loss: 7277.1357


with data augmentation

In [6]:
df_iter = DataFrameIterator(
    df, 
    input_columns=['img_path', 'img_path_extra'],
    output_columns='regression',
    image_size=(2, 2),
    batch_size=2,
    color_mode='grayscale',
    image_data_generator=ImageDataGenerator(horizontal_flip=True)
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

input_1 = Input(shape=(2, 2, 1))
input_2 = Input(shape=(2, 2, 1))
x_1 = Flatten()(input_1)
x_2 = Flatten()(input_2)
x = Concatenate()([x_1, x_2])
out = Dense(1)(x)
model = Model([input_1, input_2], out)
model.compile('adam', 'mse')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg',
             '/tmp/8.jpg',
             '/tmp/9.jpg']},
 {'column': 'img_path_extra',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg',
             '/tmp/8.jpg',
             '/tmp/9.jpg']}]

Outputs:
[{'column': 'regression',
  'mode': None,
  'values': [5, 6, 8, 7, 9, 3, 2, 1, 7, 6]}]
Epoch 1/1
5/5 [==============================] - 0s 15ms/step - loss: 19778.5533
